- https://github.com/lm-sys/FastChat/tree/main/fastchat/llm_judge
- MT-Bench：
    - 构成：一个包含80个高质量、多轮对话问题的数据集。
    - 特点：涵盖了写作、角色扮演、推理、数学、编程等8个常见类别。每个问题都包含两轮对话，旨在测试模型在多轮交互中遵循指令的能力。
    - 用途：作为一个标准化的、可控的测试环境，用于精确比较不同模型的能力。
- Types of LLM-as-a-Judge
    - Single Answer Grading
    - Pairwise Comparison
    - Reference-guided Grading
- 裁判LLM的局限性与偏见 (Limitations)
    - 位置偏见 (Position Bias)：裁判LLM倾向于偏爱第一个呈现给它的答案。实验表明，即使交换两个完全相同的答案的顺序，裁判的判断也可能发生改变。GPT-4虽然也存在此问题，但一致性（超过60%）远高于其他模型。
    - 冗长偏见 (Verbosity Bias)：裁判LLM倾向于偏爱更长、更详细的回答，即使这些回答包含不必要的重复信息。
    - 自我增强偏见 (Self-enhancement Bias)：裁判LLM可能倾向于偏爱由其自身或同系列模型生成的答案。例如，GPT-4作为裁判时，对GPT-4模型回答的胜率有约10%的提升。
    - 有限的推理与数学能力：在评判复杂的推理或数学问题时，即使裁判LLM自己有能力解决这个问题，它也容易被模型给出的错误答案误导，从而做出错误的判断。
- 应对偏见和局限性的解决方案
    - 交换位置 (Swapping Positions)：为了克服位置偏见，可以将两个模型的答案顺序交换后，再让裁判评估一次。只有在两种顺序下都判定同一个模型获胜时，才算作有效胜利，否则视为平局。
    - 思维链与参考引导 (Chain-of-Thought & Reference-guided)：为了提升评判数学和推理问题的准确性，可以采用两种策略：
        - CoT：在提示中要求裁判“首先独立地、一步一步地解决问题，然后再比较两个助手的答案”。
        - 参考引导：先让裁判LLM生成一个正确答案，然后将这个答案作为“参考”提供给它，再让它进行评判。实验证明，参考引导的方法效果显著，能将数学问题的评判失败率从70%降低到15%。
    - 处理多轮对话的提示设计：在评判多轮对话时，将完整的对话历史（两轮问答）作为一个整体输入给裁判LLM，比将两轮对话拆分成两个独立的提示要好得多。这能避免裁判因上下文缺失而做出错误判断。
- conclusion
    - 与人类高度一致：强大的LLM裁判（特别是GPT-4）与人类专家的判断一致率超过80%。这个水平与人类之间（不同的人类专家之间）的判断一致率相当。这证明了LLM-as-a-Judge作为人类评估代理的可行性。
    - GPT-4是目前最佳裁判：与其他模型（如GPT-3.5, Claude）相比，GPT-4作为裁判时表现出更高的一致性、更少的偏见和更强的判断能力。
        - LLM裁判的判断与“黄金标准”——人类专家的判断——进行直接比较。
        - 我们将两种裁判类型之间的一致性定义为，从每种类型中随机选择的（但非同一的）个体对一个随机选择的问题达成一致意见的概率。"
    - 差距越大，判断越准：当两个被评估模型的性能差距很大时，LLM裁判与人类的判断一致率接近100%。当两个模型性能接近时，一致率会下降。

In [6]:
import json
from collections import defaultdict

In [7]:
prompts = defaultdict(dict)
with open('./data/judge_prompts.jsonl', 'r') as f:
    for line in f:
        prompt = json.loads(line)
        prompts[prompt['name']] = prompt

In [30]:
len(prompt)

7

In [10]:
prompts['pair-v2']

{'name': 'pair-v2',
 'type': 'pairwise',
 'system_prompt': 'Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user\'s instructions and answers the user\'s question better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better, and "[[C]]" for a tie.

In [11]:
for name, prompt in prompts.items():
    print(name, prompt['type'], prompt['description'], prompt['category'], prompt['output_format'])

pair-v2 pairwise Prompt for general questions general [[A]]
pair-v2-multi-turn pairwise Prompt for multi-turn general questions general [[A]]
pair-math-v1 pairwise Prompt for math questions math [[A]]
pair-math-v1-multi-turn pairwise Prompt for multi-turn general questions general [[A]]
single-v1 single Prompt for general questions general [[rating]]
single-math-v1 single Prompt for general questions math [[rating]]
single-v1-multi-turn single Prompt for general questions general [[rating]]
single-math-v1-multi-turn single Prompt for general questions math [[rating]]


### single-v1

In [12]:
prompts['single-v1']

{'name': 'single-v1',
 'type': 'single',
 'system_prompt': 'You are a helpful assistant.',
 'prompt_template': '[Instruction]\nPlease act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".\n\n[Question]\n{question}\n\n[The Start of Assistant\'s Answer]\n{answer}\n[The End of Assistant\'s Answer]',
 'description': 'Prompt for general questions',
 'category': 'general',
 'output_format': '[[rating]]'}

In [14]:
# {question}, {answer}
print(prompts['single-v1']['prompt_template'])

[Instruction]
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".

[Question]
{question}

[The Start of Assistant's Answer]
{answer}
[The End of Assistant's Answer]


In [21]:
print(prompts['single-math-v1']['prompt_template'])

[Instruction]
Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider correctness and helpfulness. You will be given a reference answer and the assistant's answer. Begin your evaluation by comparing the assistant's answer with the reference answer. Identify and correct any mistakes. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".

[Question]
{question}

[The Start of Reference Answer]
{ref_answer_1}
[The End of Reference Answer]

[The Start of Assistant's Answer]
{answer}
[The End of Assistant's Answer]


### pair-v2

In [15]:
prompts['pair-v2']

{'name': 'pair-v2',
 'type': 'pairwise',
 'system_prompt': 'Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user\'s instructions and answers the user\'s question better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better, and "[[C]]" for a tie.

In [18]:
print(prompts['pair-v2']['system_prompt'])

Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. You should choose the assistant that follows the user's instructions and answers the user's question better. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of their responses. Begin your evaluation by comparing the two responses and provide a short explanation. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better, and "[[C]]" for a tie.


In [19]:
# {question}, {answer_a}, {answer_b}
print(prompts['pair-v2']['prompt_template'])

[User Question]
{question}

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]


In [25]:
print(prompts['pair-math-v1']['system_prompt'])

Please act as an impartial judge and evaluate the quality of the responses provided by two AI assistants to the user question displayed below. Your evaluation should consider correctness and helpfulness. You will be given a reference answer, assistant A's answer, and assistant B's answer. Your job is to evaluate which assistant's answer is better. Begin your evaluation by comparing both assistants' answers with the reference answer. Identify and correct any mistakes. Avoid any position biases and ensure that the order in which the responses were presented does not influence your decision. Do not allow the length of the responses to influence your evaluation. Do not favor certain names of the assistants. Be as objective as possible. After providing your explanation, output your final verdict by strictly following this format: "[[A]]" if assistant A is better, "[[B]]" if assistant B is better, and "[[C]]" for a tie.


In [26]:
print(prompts['pair-math-v1']['prompt_template'])

[User Question]
{question}

[The Start of Reference Answer]
{ref_answer_1}
[The End of Reference Answer]

[The Start of Assistant A's Answer]
{answer_a}
[The End of Assistant A's Answer]

[The Start of Assistant B's Answer]
{answer_b}
[The End of Assistant B's Answer]


### multi-turn

In [28]:
print(prompts['single-v1-multi-turn']['system_prompt'])

Please act as an impartial judge and evaluate the quality of the response provided by an AI assistant to the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, accuracy, depth, creativity, and level of detail of the response. You evaluation should focus on the assistant's answer to the second user question. Begin your evaluation by providing a short explanation. Be as objective as possible. After providing your explanation, you must rate the response on a scale of 1 to 10 by strictly following this format: "[[rating]]", for example: "Rating: [[5]]".




In [29]:
print(prompts['single-v1-multi-turn']['prompt_template'])

<|The Start of Assistant A's Conversation with User|>

### User:
{question_1}

### Assistant A:
{answer_1}

### User:
{question_2}

### Assistant A:
{answer_2}

<|The End of Assistant A's Conversation with User|>


### chat with gemini 2.5 pro

- gemini 2.5 pro
    - 原生多模态，原生 pdf 输入，
    - 精准 attention（long context 能力）
- 带着问题，query；
- 不断追问；
- 定位到 paper 相关 section；